In [86]:
# API KEY: sk-82F0fUi3hSi_drnaRG9Z72jEbFbIqPdYHHClASd7kET3BlbkFJJqxRZ40hP-p3XxjBMCQK3qUT40AmziQLi9Vz5WIcIA
# ORG ID: org-rjJCwF3sTsfOnw3I2Eaj4zdP
import json
import re
import math
import pandas as pd
from algorithms import *
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

#### Necessary pip installation commands include:
- pip install pandas
- pip install nltk
#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key


In [87]:
def combined_randomized_preprocessing(data_one, data_two):
    data_one_pre = toxicity_challenge_preprocess(data_one)
    data_two_pre = toxicity_challenge_preprocess(data_two)
    print(f"Toxic: {len(data_one_pre[data_one_pre['Toxic'] == 1])}")
    print(f"Non-Toxic: {len(data_one_pre[data_two_pre['Toxic'] == 0])}")
    complete_data = pd.concat([data_one_pre, data_two_pre]).sample(frac=1, random_state=42).reset_index(drop=True)

    return complete_data

In [88]:
# reads kaggle toxic classification challenge dataset
df = pd.read_csv('../data/csv/train.csv')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [89]:
len(df[df['toxic'] == 1])

15294

In [90]:
len(df[df['toxic'] == 0])

144277

In [91]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [92]:
training_size = 5000
evaluation_size = 500
container = 1500
toxic_set = df[df['toxic'] == 1 &
    ~df['comment_text'].apply(check_repetition)
].sample(n=(training_size + container),random_state=42)
non_toxic_set = df[df['toxic'] == 0 &
    ~df['comment_text'].apply(check_repetition)
].sample(n=(training_size + container),random_state=42)

In [93]:
toxic_train, toxic_eval = toxic_set[:training_size], toxic_set[training_size:].sample(frac=1, random_state=42).reset_index(drop=True)
non_toxic_train, non_toxic_eval = non_toxic_set[:training_size], non_toxic_set[training_size:].sample(frac=1, random_state=42).reset_index(drop=True)

In [94]:
toxic_eval[toxic_eval['toxic'] == 1].iloc[:500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,d313e1d54145d91f,"Suck COck!!!\r\nyou do!!!! you do!!!! 05:33, 1...",1,0,1,0,1,0
1,8a2535680729c7ec,NAZIS WHO RUN WIKI WILL NOT EVEN ALLOW ME TO C...,1,0,0,0,0,0
2,ee0f6fcaa55010cf,Thank you for being such a politically correct...,1,0,1,0,1,0
3,d3f844a0e32d8bce,also do not vandilise streets when you feel li...,1,0,0,0,0,1
4,314bdb4e49031b2b,"Your the thickest, im still in school, and i b...",1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
566,511d110432c4835a,My source was fucking TELEVISED.,1,0,0,0,0,0
567,9cfde0f70ab733d1,", or else I'll cure your virginity from the wr...",1,0,1,0,0,0
568,3c622184626caedd,oh why dear johnny boy part three \r\n\r\nshit...,1,0,1,0,0,0
569,3cf4db7dcb532ea6,before calling me a bitch on my own talk at le...,1,0,1,0,1,1


In [95]:
non_toxic_eval[non_toxic_eval['toxic'] == 0].iloc[:500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,5043a670a01d8337,"""\r\nSaying """"impose neoliberal orthodoxy"""" is...",0,0,0,0,0,0
1,8a065dc3492526f7,But others are ok.,0,0,0,0,0,0
2,0772169fd96b7d38,Physical characteristics/ Origins/ Genetics \r...,0,0,0,0,0,0
3,785329727911959e,"""==GA Review==\r\n\r\n:This review is transclu...",0,0,0,0,0,0
4,645e8418b97f390d,"I have read it, and I continue to see that Mr....",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
495,d50ed2c9f45ff553,OK. The list of Gnostic Saints comes from a bo...,0,0,0,0,0,0
496,7c9a6840f119e00b,I knew I should have gotten a superinjunction ...,0,0,0,0,0,0
497,c80f9b0bd1e45660,hensel pic==\r\nThe issue isn't who the pictur...,0,0,0,0,0,0
498,8c14a680eec0c3b0,"Yes, that how I took it, and you can understan...",0,0,0,0,0,0


In [96]:
toxic_train[toxic_train['toxic'] == 1].iloc[:2500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
145663,2040644271b7cd54,shut your mouth.........you grabage dump,1,0,0,0,1,0
133605,cac3c5d735c3e2ee,"it is a constructive edit you idiot, every kid...",1,0,1,0,1,0
79118,d3c53368eac8f6e1,"SlimVirgin, show me the racial insult that I h...",1,0,0,0,0,0
57756,9a90d25059773747,"Cumulus Cloud: get a life, you loser \r\n\r\nC...",1,0,0,0,0,0
67941,b5c0e400c85e6bcf,i hate lewis and clark,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
38037,6586365e7f2f424e,== T never said i did you ugly bastard. Get o...,1,0,1,0,1,0
23736,3eb303deae78cfb7,You are kidding me ? Look at the previous vers...,1,0,0,0,0,0
105837,3639e088a5104198,Fraggle81 \r\nYou're a faggot.,1,0,1,0,1,1
46959,7d79a7b66e147247,"hey Bishonen \r\nHey dude,\r\n\r\nthanks for y...",1,0,0,0,0,0


In [97]:
non_toxic_train[non_toxic_train['toxic'] == 0].iloc[:2500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
145490,1d8d7a2ede11c571,"""\r\nNo, it doesn't do fill-in. In handles """"p...",0,0,0,0,0,0
154734,b27d83ffac049c23,"Well, if The Tartan, a sometime student public...",0,0,0,0,0,0
79795,d57e18ce59b8cee5,"hi, friends, should we talk about the line mar...",0,0,0,0,0,0
151445,7d1f2a94425ce1ab,"""\r\nYes, my cat was subject to attempt to tes...",0,0,0,0,0,0
40658,6c873af205a65db2,2012 (UTC)\r\nTry The Books of the Bible. 16...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
12774,21e1cdfd0285612c,Semi-protected edit request on 25 January 2014...,0,0,0,0,0,0
46653,7cafb3e8c88ab2aa,I actually did intentionally remove the sub-br...,0,0,0,0,0,0
62189,a65d4d1840131c2b,"""\r\n\r\nThe """"Objections and replies"""" sectio...",0,0,0,0,0,0
66298,b15db678597ca235,Do not remove my accurate commentary \r\n\r\nd...,0,0,0,0,0,0


In [98]:
full_training_set = combined_randomized_preprocessing(toxic_train[toxic_train['toxic'] == 1].iloc[:2500], non_toxic_train[non_toxic_train['toxic'] == 0].iloc[:2500])
full_evaluation_set = combined_randomized_preprocessing(toxic_eval[toxic_eval['toxic'] == 1].iloc[:500], non_toxic_eval[non_toxic_eval['toxic'] == 0].iloc[:500])


Toxic: 2500
Non-Toxic: 2500
Toxic: 500
Non-Toxic: 500


In [99]:
full_evaluation_set

,Text,Toxic
0,believe section Polynomials Maximal LFSRs remo...,0
1,best source Dzierzon biography L Brożek Jan Dz...,0
2,First cover current run Batista cover second i...,0
3,Reviewer Hello account granted reviewer userri...,0
4,Either wikipedia letting anyone become adminis...,1
...,...,...
995,didnt say anything fight Niteshift need put ch...,1
996,P4k Wikipedia delete things people worked worl...,1
997,Shii act intimidation concept intimidation mea...,0
998,would also like say masturbate also gay lol,1


In [107]:
len(full_evaluation_set[full_evaluation_set['Toxic'] == 1])

500

In [105]:
len(full_evaluation_set[full_evaluation_set['Toxic'] == 0])

500

In [101]:
len(full_training_set[full_training_set['Toxic'] == 0])

2500

In [102]:
len(full_training_set[full_training_set['Toxic'] == 1])

2500

In [103]:
# saves preprocessed fine_tuning data for later usage
full_training_set.to_csv('../data/csv/toxic_classification_training_set.csv', index=False)

In [104]:
# saves preprocessed evaluation data to csv file
full_evaluation_set.to_csv('../data/csv/toxic_classification_evaluation_set.csv', index=False)